<a href="https://colab.research.google.com/github/mamad321/Machine-learning-hw/blob/main/Week7_Distributed_XGBoost_with_Ray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install ray


     |████████████████████████████████| 54.7 MB 32 kB/s 
     |████████████████████████████████| 118 kB 50.5 MB/s 


In [2]:
pip install xgboost_ray

     |████████████████████████████████| 136 kB 5.2 MB/s 


In [3]:
pip install sklearn

In [4]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

train_x, train_y = load_breast_cancer(return_X_y=True)
train_set = RayDMatrix(train_x, train_y)

evals_result = {}
bst = train(
    {
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
    },
    train_set,
    evals_result=evals_result,
    evals=[(train_set, "train")],
    verbose_eval=False,
    ray_params=RayParams(num_actors=2, cpus_per_actor=1))

bst.save_model("model.xgb")
print("Final training error: {:.4f}".format(
    evals_result["train"]["error"][-1]))

/usr/local/lib/python3.7/dist-packages/redis/connection.py:77: UserWarning: redis-py works best with hiredis. Please consider installing
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:166: UserWarning: You are using `xgboost_ray` with a legacy XGBoost version (version 0.90). While we try to support older XGBoost versions, please note that this library is only fully tested and supported for XGBoost >= 1.4. Please consider upgrading your XGBoost version (`pip install -U xgboost`).
  warnings.warn(LEGACY_WARNING)
(raylet) /usr/local/lib/python3.7/dist-packages/redis/connection.py:77: UserWarning: redis-py works best with hiredis. Please consider installing
(raylet)   warnings.warn(msg)
2021-11-17 11:28:14,788	INFO main.py:971 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
(raylet) /usr/local/lib/python3.7/dist-packages/redis/connection.py:77: UserWarning: redis-py works best with hiredis. Please consider

Final training error: 0.0053


In [5]:
from xgboost_ray import RayDMatrix, RayParams, predict
from sklearn.datasets import load_breast_cancer
import xgboost as xgb

data, labels = load_breast_cancer(return_X_y=True)

dpred = RayDMatrix(data, labels)

bst = xgb.Booster(model_file="model.xgb")
pred_ray = predict(bst, dpred, ray_params=RayParams(num_actors=2))

print(pred_ray)

/usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:166: UserWarning: You are using `xgboost_ray` with a legacy XGBoost version (version 0.90). While we try to support older XGBoost versions, please note that this library is only fully tested and supported for XGBoost >= 1.4. Please consider upgrading your XGBoost version (`pip install -U xgboost`).
  warnings.warn(LEGACY_WARNING)
2021-11-17 11:28:21,749	INFO main.py:1535 -- [RayXGBoost] Created 2 remote actors.
(raylet) /usr/local/lib/python3.7/dist-packages/redis/connection.py:77: UserWarning: redis-py works best with hiredis. Please consider installing
(raylet)   warnings.warn(msg)
(raylet) /usr/local/lib/python3.7/dist-packages/redis/connection.py:77: UserWarning: redis-py works best with hiredis. Please consider installing
(raylet)   warnings.warn(msg)
2021-11-17 11:28:23,975	INFO main.py:1552 -- [RayXGBoost] Starting XGBoost prediction.


[0.07511145 0.04715708 0.02642454 0.11464816 0.08779665 0.19609766
 0.02642454 0.03200296 0.03968306 0.06930447 0.09290799 0.02642454
 0.03476055 0.05490047 0.05012454 0.02642454 0.03337562 0.02642454
 0.02642454 0.9528658  0.9751126  0.9751126  0.07511145 0.02642454
 0.02642454 0.04102892 0.03118352 0.02642454 0.02642454 0.08779665
 0.02642454 0.03476055 0.02642454 0.02642454 0.02642454 0.02642454
 0.0665335  0.9690314  0.33619487 0.0696831  0.5729184  0.15662025
 0.03451282 0.03200296 0.12301622 0.04102892 0.9751126  0.04096693
 0.97315884 0.9234187  0.9751126  0.9751126  0.9751126  0.03059936
 0.09855346 0.9751126  0.02642454 0.03755146 0.9751126  0.97315884
 0.9751126  0.97315884 0.02642454 0.9751126  0.03120249 0.03120249
 0.97315884 0.9751126  0.8630876  0.9751126  0.02642454 0.9751126
 0.02642454 0.18486539 0.9751126  0.02642454 0.9684864  0.07511145
 0.04528174 0.9751126  0.97315884 0.8423582  0.02642454 0.02642454
 0.9751126  0.02642454 0.18525673 0.02642454 0.9610139  0.91992

In [6]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

num_actors = 1
num_cpus_per_actor = 1

ray_params = RayParams(
    num_actors=num_actors, cpus_per_actor=num_cpus_per_actor)

def train_model(config):
    train_x, train_y = load_breast_cancer(return_X_y=True)
    train_set = RayDMatrix(train_x, train_y)

    evals_result = {}
    bst = train(
        params=config,
        dtrain=train_set,
        evals_result=evals_result,
        evals=[(train_set, "train")],
        verbose_eval=False,
        ray_params=ray_params)
    bst.save_model("model.xgb")

from ray import tune

# Specify the hyperparameter search space.
config = {
    "tree_method": "approx",
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"],
    "eta": tune.loguniform(1e-4, 1e-1),
    "subsample": tune.uniform(0.5, 1.0),
    "max_depth": tune.randint(1, 9)
}

# Make sure to use the `get_tune_resources` method to set the `resources_per_trial`
analysis = tune.run(
    train_model,
    config=config,
    metric="train-error",
    mode="min",
    num_samples=4,
    resources_per_trial=ray_params.get_tune_resources())
print("Best hyperparameters", analysis.best_config)

2021-11-17 11:28:28,295	WARNING function_runner.py:564 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2021-11-17 11:28:28,321	INFO logger.py:606 -- pip install 'ray[tune]' to see TensorBoard files.
2021-11-17 11:28:28,322	WARNING callback.py:115 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


Trial name,status,loc,eta,max_depth,subsample
train_model_7c55a_00000,PENDING,,0.000945469,3,0.857978
train_model_7c55a_00001,PENDING,,0.0102518,5,0.673034
train_model_7c55a_00002,PENDING,,0.00314708,3,0.663506
train_model_7c55a_00003,PENDING,,0.000731633,5,0.6633


(raylet) /usr/local/lib/python3.7/dist-packages/redis/connection.py:77: UserWarning: redis-py works best with hiredis. Please consider installing
(raylet)   warnings.warn(msg)
(pid=404) /usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:166: UserWarning: You are using `xgboost_ray` with a legacy XGBoost version (version 0.90). While we try to support older XGBoost versions, please note that this library is only fully tested and supported for XGBoost >= 1.4. Please consider upgrading your XGBoost version (`pip install -U xgboost`).
(pid=404)   warnings.warn(LEGACY_WARNING)
(pid=404) /usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:422: UserWarning: `num_actors` in `ray_params` is smaller than 2 (1). XGBoost will NOT be distributed!
(pid=404)   f"`num_actors` in `ray_params` is smaller than 2 "
(pid=404) 2021-11-17 11:28:30,747	INFO main.py:971 -- [RayXGBoost] Created 1 new actors (1 total actors). Waiting until actors are ready for training.
(raylet) /usr/local/lib/pyt

Trial name,status,loc,eta,max_depth,subsample
train_model_7c55a_00000,RUNNING,172.28.0.2:404,0.000945469,3,0.857978
train_model_7c55a_00001,PENDING,,0.0102518,5,0.673034
train_model_7c55a_00002,PENDING,,0.00314708,3,0.663506
train_model_7c55a_00003,PENDING,,0.000731633,5,0.6633


Result for train_model_7c55a_00000:
  date: 2021-11-17_11-28-34
  done: false
  experiment_id: 77317c7ffb9a4314ad7397e661af46dc
  hostname: d2a4958c047c
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  pid: 404
  time_since_restore: 3.813161849975586
  time_this_iter_s: 3.813161849975586
  time_total_s: 3.813161849975586
  timestamp: 1637148514
  timesteps_since_restore: 0
  train-error: 0.043937
  train-logloss: 0.692345
  training_iteration: 1
  trial_id: 7c55a_00000
  
Result for train_model_7c55a_00000:
  date: 2021-11-17_11-28-34
  done: true
  experiment_id: 77317c7ffb9a4314ad7397e661af46dc
  experiment_tag: 0_eta=0.00094547,max_depth=3,subsample=0.85798
  hostname: d2a4958c047c
  iterations_since_restore: 10
  node_ip: 172.28.0.2
  pid: 404
  time_since_restore: 3.8939931392669678
  time_this_iter_s: 0.006039142608642578
  time_total_s: 3.8939931392669678
  timestamp: 1637148514
  timesteps_since_restore: 0
  train-error: 0.024605
  train-logloss: 0.685147
  training_iterat

...

(raylet) /usr/local/lib/python3.7/dist-packages/redis/connection.py:77: UserWarning: redis-py works best with hiredis. Please consider installing
(raylet)   warnings.warn(msg)
(pid=720) 2021-11-17 11:28:51,460	INFO main.py:1016 -- [RayXGBoost] Starting XGBoost training.


Result for train_model_7c55a_00003:
  date: 2021-11-17_11-28-52
  done: false
  experiment_id: 06ecab3ab82e42fa902d53d3c63ccf83
  hostname: d2a4958c047c
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  pid: 720
  time_since_restore: 3.9229514598846436
  time_this_iter_s: 3.9229514598846436
  time_total_s: 3.9229514598846436
  timestamp: 1637148532
  timesteps_since_restore: 0
  train-error: 0.043937
  train-logloss: 0.692526
  training_iteration: 1
  trial_id: 7c55a_00003
  
Result for train_model_7c55a_00003:
  date: 2021-11-17_11-28-52
  done: true
  experiment_id: 06ecab3ab82e42fa902d53d3c63ccf83
  experiment_tag: 3_eta=0.00073163,max_depth=5,subsample=0.6633
  hostname: d2a4958c047c
  iterations_since_restore: 10
  node_ip: 172.28.0.2
  pid: 720
  time_since_restore: 4.019043445587158
  time_this_iter_s: 0.007227182388305664
  time_total_s: 4.019043445587158
  timestamp: 1637148532
  timesteps_since_restore: 0
  train-error: 0.019332
  train-logloss: 0.686892
  training_iterat

Trial name,status,loc,eta,max_depth,subsample,iter,total time (s),train-logloss,train-error
train_model_7c55a_00000,TERMINATED,172.28.0.2:404,0.000945469,3,0.857978,10,3.89399,0.685147,0.024605
train_model_7c55a_00001,TERMINATED,172.28.0.2:508,0.0102518,5,0.673034,10,3.86645,0.612398,0.019332
train_model_7c55a_00002,TERMINATED,172.28.0.2:614,0.00314708,3,0.663506,10,3.96482,0.667255,0.024605
train_model_7c55a_00003,TERMINATED,172.28.0.2:720,0.000731633,5,0.6633,10,4.01904,0.686892,0.019332


(pid=720) 2021-11-17 11:28:52,626	INFO main.py:1498 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 4.06 seconds (1.16 pure XGBoost training time).
2021-11-17 11:28:52,791	INFO tune.py:630 -- Total run time: 24.50 seconds (24.30 seconds for the tuning loop).


Best hyperparameters {'tree_method': 'approx', 'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'eta': 0.010251810127980175, 'subsample': 0.6730339063089048, 'max_depth': 5}
